# **ChatGPT(LangChain)** #
本レポートではChatGPTに加えてLangChainを用いることで、段階的な処理を行いながら回答
を生成するBotの作成を目指す。今回は最新のAI関連ニュースについての応答を生成するBotを題材にする。

1. 決まった一つの処理を行うモデル
まず、AI専門ニュースサイト (https://aismiley.co.jp/ai-news_category/generative-ai/) から単一の質問に回答するBotを作成する。
初めに、必要なライブラリをインストールする。

In [22]:
#!pip install openai langchain llama-index google-search-results faiss-cpu
#!pip install nbconvert jupyter_contrib_nbextensions jupyter_nbextensions_configurator notebook
#!pip install --upgrade notebook==6.4.12

In [23]:
import os
# 環境変数をセットする
os.environ["OPENAI_API_KEY"] = ""
os.environ["SERP_API_KEY"] = ""

SERP_API_KEY = os.environ["SERP_API_KEY"]

まず、AI専門ニュースサイト (https://aismiley.co.jp/ai-news_category/generative-ai/) から文書を取得し、その文書を一定数のトークンの集まりのチャンクに分解する。

In [24]:
import re
from langchain.document_loaders import WebBaseLoader
# ニュースサイトからテキストを取得する
loader = WebBaseLoader(
"https://aismiley.co.jp/ai-news_category/generative-ai/"
)
documents= loader.load()
# 簡易的な前処理を行う
documents[0].page_content = re.sub(r"\n+", "\n", documents[0].page_content)

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# テキストスプリッターを初期化する
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
# テキストをチャンクに分割する
texts = text_splitter.split_documents(documents)

print(texts[0])

page_content='生成AI ｜AIsmiley AIニュース｜DXを推進するAIポータルメディア「AIsmiley」\n \n \nDXを推進するAIポータルメディア「AIsmiley」｜ AI製品・サービスの比較・検索サイト\nサイトの使い方\n情報提供はこちらから\n掲載希望の企業様へ\n\r\n                                                            ChatGPT連携サービス                            \nChatGPT連携サービスTOP\nチャットボット\nカスタマー\nライティング\nマーケティング\nテキスト要約\n議事録作成\n導入支援\nリスキリング\n翻訳\nプログラミング' metadata={'source': 'https://aismiley.co.jp/ai-news_category/generative-ai/', 'title': '生成AI ｜AIsmiley AIニュース｜DXを推進するAIポータルメディア「AIsmiley」', 'language': 'ja'}


埋め込みを用いてクエリに最も関連するチャンクを抽出するRetrieverを作成する。

In [26]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
# 埋め込みを初期化する
embeddings = OpenAIEmbeddings()
# ベクターストアに文書と埋め込みを格納
db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever()

Retrieverを用いて、入力されたクエリに関連する該当部分を参照し回答させる。

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
# LLM ラッパーを読み込む
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
# チェーンを作り、それを使って質問に答える
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    output_key="description_str")

question= "最も新しいニュースは何ですか？"
print(qa_chain.run(question))

最も新しいニュースは、Rapidusがシリコンバレーに新会社を設立し、AI半導体関連の顧客開拓を推進するというものです。


# **2. 決まった複数の処理を行うBot** #
次に、AI関連ニュースサイトから質問に回答し、その情報の詳細を要約するBotを作成する。
まず、最新ニュース情報を入力として、それを詳細に要約するChainを作成する。

In [28]:
from langchain import LLMChain, PromptTemplate
day_qa_template_ja = """文脈情報は以下の通りです。

---------------------
{description_str}
---------------------
以前の知識を用いず、この文脈情報で以下の問いに答えてください。:\

文脈情報で言及されたものを詳しく要約してください。
"""
day_qa_prompt_template = PromptTemplate(
input_variables=["description_str"],
template=day_qa_template_ja,
)
day_qa_chain = LLMChain(llm=llm,
                        prompt=day_qa_prompt_template,
                        output_key="answer_str")

その後、(1)で作成したサイト内で最も新しいニュースを答えるチェーンとその情報を要約させるチェーンを結合する。

In [29]:
from langchain.chains import SimpleSequentialChain
garbage_qa_chain = SimpleSequentialChain(chains=[qa_chain, day_qa_chain],

verbose=True)


In [30]:
garbage_qa_chain.run(question)



> Entering new SimpleSequentialChain chain...
最も新しいニュースは、Rapidusがシリコンバレーに新会社を設立し、AI半導体関連の顧客開拓を推進するという内容です。
Rapidusがシリコンバレーに新会社を設立し、AI半導体関連の顧客開拓を推進するというニュースが最新である。

> Finished chain.


'Rapidusがシリコンバレーに新会社を設立し、AI半導体関連の顧客開拓を推進するというニュースが最新である。'

# **3. 自由に与えられたツールを使って回答するBot** #
最後に、インターネットを自由に検索し、最新のニュースとその要約を回答するBotを作成する。
このためにAgentと呼ばれる機能を利用する。
まず、インターネットを検索する機能を利用するため、SerpAPI https://serpapi.com/ のAPIキーを発行する。

次に、エージェントが利用するツールを定義する

In [31]:
from langchain import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
# 検索するAPIのラッパーを用意する
search = SerpAPIWrapper(
serpapi_api_key=SERP_API_KEY,
params = {"engine": "bing","gl": "jp","hl": "ja"}
)

"""
list = []
def memorize():
    list.append("{summarizedText}")
    return
"""

# ツールにエージェントが利用する機能を格納する
tools = [
Tool(
name="Search",
func=search.run,
description="useful for when you need to answer questions what details about input text"
),
Tool(
    name="Summarize",
    func=day_qa_chain.run,
    description="useful for summarizing text you have searched for"
)
]

次にエージェントがどのように思考するかを示すプロンプトを作成する
備考:安定した推論を行うため、英語でのプロンプトにしている

In [32]:
# エージェントが利用するプロンプトを作成する
template = """Must Answer the following questions.You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer (in Japanese): the final answer to the original input question in Japanese
Question: {input}
{agent_scratchpad}"""

In [33]:
from langchain.prompts import BaseChatPromptTemplate
from langchain.schema import HumanMessage
from typing import List, Union
# エージェントのプロンプトのテンプレートを定義する
class CustomPromptTemplate(BaseChatPromptTemplate):
    template: str
    tools: List[Tool]
    def format_messages(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)



エージェントの出力を追跡するパーサーという機能を作成する

In [34]:
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish, HumanMessage
# エージェントの出力のパーサーを作成する
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Final Answer (in Japanese):" in llm_output:
            return AgentFinish(
                return_values={
                "output": llm_output.split("Final Answer (in Japanese):")[-1].strip()
                },
            log=llm_output,
            )
        # 行動と行動のための入力をパースする
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # 行動と行動のための入力を返す
        return AgentAction(tool=action,
            tool_input=action_input.strip(" ").strip('"'),
            log=llm_output)

output_parser = CustomOutputParser()

以上のものを用いて、自由にツールを用いて与えられたタスクを解こうとするエージェントを設定する

In [35]:
from langchain.agents import LLMSingleActionAgent
from langchain.agents import AgentExecutor
# エージェントを設定する
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
llm_chain=llm_chain,
output_parser=output_parser,
stop=["\nObservation:"],
allowed_tools=tool_names
)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,

tools=tools,
verbose=True)

In [36]:
agent_executor.run("最新のAI関連の技術革新について3つほど調べ、それぞれを要約してください。")



> Entering new AgentExecutor chain...
Thought: I need to search for the latest AI-related technological innovations
Action: Search
Action Input: Latest AI technological innovations

Observation:['Artificial intelligence. AI for everything: 10 Breakthrough Technologies 2024. Generative AI tools like ChatGPT reached mass adoption in record time, and reset the course of an entire industry....', 'Our four big bets for 2023 were that the next big thing in chatbots would be multimodal (check: the most powerful large language models out there, OpenAI’s GPT …', 'August 17, 2023. Contributor: Lori Perri. Innovations in and around generative AI dominate and have transformative impact. The 2023 Gartner Hype Cycle™ for Artificial …', 'The 3 Most Important AI Innovations of 2023. 8 minute read. This video cannot be played because of a technical error. (Error Code: 102006) By Billy Perrigo. …', 'Innovations here include physics-informed AI, composite AI, causal AI, generative AI, foundation models

'2023年のAI関連の技術革新には、生成AI、複合AI、因果AI、基礎モデル、深層学習などが含まれており、特に生成AIの進化が注目されています。'

（３）わかったこと・わからなかったこと（この講義で解消したいこと）
    わかったこと：生成AIは様々なサービスやAPIと連携してこそ真価を発揮するということ

    わからなかったこと：同じプロンプトでも出力ごとに大きな差ができており、それを抑えるためのプロンプトエンジニアリングが難しかった